# Auswertung und Visualisierung der erhobenen Daten

##### Quellen
```
Pandas. (2025, 7. Juli). Pandas Documentation. Abgerufen am 11. Juli 2025, von https://pandas.pydata.org/docs/


Plotly. (o. D.). Plotly Express in Python. Abgerufen am 16. Juli 2025, von https://plotly.com/python/plotly-express/
```


In [2]:
import pandas as pd

In [3]:
# Bewertungen der Systemantworten im Originalkontext für beide durchläufe einlesen

df_original = pd.read_csv("../llm_bewertung_originalkontext_beide_durchgänge/bewertung_original.csv", index_col=0)
df_original_2 = pd.read_csv("../llm_bewertung_originalkontext_beide_durchgänge/bewertung_original_2.csv", index_col=0)

# Bewertungen der Systemantworten im manipulierten Kontext für beide durchläufe einlesen

df_manipuliert = pd.read_csv("../llm_bewertungen_manipulierter_kontext_beide_durchgänge/bewertung_manipulierter_kontext.csv", index_col=0)
df_manipuliert_2 = pd.read_csv("../llm_bewertungen_manipulierter_kontext_beide_durchgänge/bewertung_manipulierter_kontext_2.csv", index_col=0)

# Manuelle bewertung einlesen
df_manuell = pd.read_csv("../manuelle_evaluation_csv/manuelle_bewertung_manipuliert_2.csv", index_col=0)

### Manuelle Bewertung mit der des LLMs vergleichen

In [6]:
def vergleich_bewertungen(df_manuell, df_llm):
    
    # Relevante daten aus dem mannuellen bewertungsdF herausziehen
    vergleich_df = df_manuell[["frage", "fake_news_art", "system", "antwort", "manuelle_bewertung"]].copy()

    vergleich_df["system"] = vergleich_df["system"].str.extract(r"^(vector_rag|graph_rag|hybrid_rag)")


    # leerer Df 
    df_gesamt = pd.DataFrame()

    for system in ["vector_rag", "graph_rag", "hybrid_rag"]:
        spalte_antwort = f"{system}_antwort_manipuliert_2"
        spalte_bewertung = f"{system}_antwort_manipuliert_bewertung_2"

        # Extrahiert die Frage, Fake News Art, Antwort und die Bewertung des llms
        df_temp = df_llm[["frage", "fake_news_art", spalte_antwort, spalte_bewertung]].copy()
        df_temp["system"] = system


        df_temp["antwort"] = df_temp[spalte_antwort]
        df_temp["llm_bewertung"] = df_temp[spalte_bewertung]

        df_gesamt = pd.concat([df_gesamt, df_temp[["frage", "fake_news_art", "system", "antwort", "llm_bewertung"]]], ignore_index=True)

    # mergen
    merged_df = pd.merge(
        vergleich_df, df_gesamt,
        on=["frage", "fake_news_art", "antwort", "system"],
        how="left"
    )

    # abweichungen markieren
    merged_df["abweichung"] = merged_df["manuelle_bewertung"] != merged_df["llm_bewertung"]

    return merged_df

In [7]:
x_abweichungen = vergleich_bewertungen(df_manuell, df_manipuliert_2)

In [9]:
x_abweichungen

,frage,fake_news_art,system,antwort,manuelle_bewertung,llm_bewertung,abweichung
0,Wie viele Teilnehmer wurden tatsächlich bei de...,Fake Fact,vector_rag,"Bei dem Autokorso in Tucson, der von Jorge org...",1,1,False
1,Wie hoch war der Anteil der Wähler in Kaliforn...,Fake Fact,vector_rag,Bei der aktuellen Wahl haben 40 Prozent der Wä...,1,1,False
2,Welche finanziellen Auswirkungen könnten Trump...,Fake Fact,vector_rag,Gemäß den Modellierungen des Instituts der deu...,0,0,False
3,Welche Steuerpolitik verfolgen Harris und Trum...,Fake Fact,vector_rag,"Beide Präsidentschaftskandidaten, Kamala Harri...",1,1,False
4,Wer wurde von Trump für das Amt des Justizmini...,Fake Fact,vector_rag,Donald Trump hat den erzkonservativen Abgeordn...,1,1,False
...,...,...,...,...,...,...,...
58,Wie würde sich die Situation der deutschen Aut...,Exaggeration,hybrid_rag,Unter einem Präsidenten Donald Trump würde sic...,1,1,False
59,Wie bewerten Ökonomen Trumps wirtschaftliche P...,Exaggeration,hybrid_rag,Ökonomen bewerten Trumps wirtschaftliche Pläne...,1,1,False
60,Wie wurde die Nominierung von Gaetz zum Justiz...,Exaggeration,hybrid_rag,Der bereitgestellte Kontext enthält keine spez...,0,0,False
61,Welche maximale Höhe der Strafzölle drohte Don...,Exaggeration,hybrid_rag,Der bereitgestellte Kontext enthält keine spez...,0,0,False


In [10]:
x_abweichungen["abweichung"].describe()

count        63
unique        2
top       False
freq         58
Name: abweichung, dtype: object

In [ ]:
#x_abweichungen.to_csv("abweichungen_llm_human.csv")

## cohens kappa-koeffizient berechnen

In [147]:
from sklearn.metrics import cohen_kappa_score


kappa = cohen_kappa_score(x_abweichungen["manuelle_bewertung"], x_abweichungen["llm_bewertung"])
print(f"Cohen's Kappa: {kappa:.4f}")

Cohen's Kappa: 0.8087




**cohens Kappa =**  0.8087

## Agreement: 58 / 63  
**Agreement =** 92.06%  



### Daten aus beiden Durchgängen aggregieren

In [11]:

# Spalten mit den bewertungen für die Antworten im manipulierten Kontext aus Durchgang 1
bewertungspalten = [
    "vector_rag_antwort_manipuliert_bewertung", 
    "graph_rag_antwort_manipuliert_bewertung",
    "hybrid_rag_antwort_manipuliert_bewertung"
]

# Dictionary für spätere Zusammenführung
erg = {}

# über die bewertungspalten iterieren
for spalte in bewertungspalten:
    # für den 2. Durchgang suffix hinzufügen
    spalte_2 = spalte + "_2"
    # Für spätere Umbennung _bewertung aus spaltennamen entfernen
    col_prefix = spalte.replace("_bewertung", "")

    # Beide Dfs vereinheitlichen
    df1 = df_manipuliert[["fake_news_art", spalte]].rename(columns={spalte: "bewertung"})
    df2 = df_manipuliert_2[["fake_news_art", spalte_2]].rename(columns={spalte_2: "bewertung"})

    #dfs konkatenieren
    df_kombi = pd.concat([df1, df2], ignore_index=True)

    # Zählen der 0er und 1er je Fake news Art
    counts = df_kombi.groupby(["fake_news_art", "bewertung"]).size().unstack(fill_value=0)
    
    # Umbenennen der Spalten
    counts.columns = [f"{col_prefix}_bewertung_0", f"{col_prefix}_bewertung_1"]
    erg[spalte] = counts

# Zu einem df zusammenfügen
df_erg_manip = pd.concat(erg.values(), axis=1)

# Summe hinzufüen
df_erg_manip.loc["Summe"] = df_erg_manip.sum(numeric_only=True)


In [16]:

# Spalten mit den bewertungen für die Antworten im originalkontext aus Durchgang 1
bewertungspalten = [
    "vector_rag_antwort_original_bewertung", 
    "graph_rag_antwort_original_bewertung",
    "hybrid_rag_antwort_original_bewertung"
]

# Dictionary für spätere Zusammenführung
erg = {}

# über die bewertungspalten iterieren
for spalte in bewertungspalten:
    # für den 2. Durchgang suffix hinzufügen
    spalte_2 = spalte + "_2"
    # Für spätere Umbennung _bewertung aus spaltennamen entfernen
    col_prefix = spalte.replace("_bewertung", "")

    # Beide Dfs vereinheitlichen
    df1_or = df_original[["fake_news_art", spalte]].rename(columns={spalte: "bewertung"})
    df2_or = df_original_2[["fake_news_art", spalte_2]].rename(columns={spalte_2: "bewertung"})

    # dfs konkatenieren
    df_kombi_or = pd.concat([df1_or, df2_or], ignore_index=True)

    # Zählen der 0er und 1er 
    counts = df_kombi_or.groupby(["fake_news_art", "bewertung"]).size().unstack(fill_value=0)

    # Umbennen der Spalten
    counts.columns = [f"{col_prefix}_bewertung_0", f"{col_prefix}_bewertung_1"]
    erg[spalte] = counts

# Zu einem Df zusammenführen
df_erg_org = pd.concat(erg.values(), axis=1)

# Summe hinzufügen
df_erg_org.loc["Summe"] = df_erg_org.sum(numeric_only=True)


In [17]:
df_erg_org

,vector_rag_antwort_original_bewertung_0,vector_rag_antwort_original_bewertung_1,graph_rag_antwort_original_bewertung_0,graph_rag_antwort_original_bewertung_1,hybrid_rag_antwort_original_bewertung_0,hybrid_rag_antwort_original_bewertung_1
fake_news_art,,,,,,
Exaggeration,51,139,132,58,43,147
Fake Fact,32,158,129,61,31,159
Framing,30,160,127,63,25,165
Summe,113,457,388,182,99,471


In [14]:
df_erg_manip

,vector_rag_antwort_manipuliert_bewertung_0,vector_rag_antwort_manipuliert_bewertung_1,graph_rag_antwort_manipuliert_bewertung_0,graph_rag_antwort_manipuliert_bewertung_1,hybrid_rag_antwort_manipuliert_bewertung_0,hybrid_rag_antwort_manipuliert_bewertung_1
fake_news_art,,,,,,
Exaggeration,72,118,127,63,71,119
Fake Fact,60,130,137,53,68,122
Framing,73,117,124,66,67,123
Summe,205,365,388,182,206,364


## Df mit den absoluten Resilience Rates erstellen

Formel für Berechnung in der BA

**nach Fake news Art:** 

In [17]:
df_resilience_rate_by_type = pd.DataFrame({
    "VectorRAG": df_erg_manip["vector_rag_antwort_manipuliert_bewertung_1"] / 
                 (df_erg_manip["vector_rag_antwort_manipuliert_bewertung_0"] + df_erg_manip["vector_rag_antwort_manipuliert_bewertung_1"]),
    
    "GraphRAG": df_erg_manip["graph_rag_antwort_manipuliert_bewertung_1"] / 
                (df_erg_manip["graph_rag_antwort_manipuliert_bewertung_0"] + df_erg_manip["graph_rag_antwort_manipuliert_bewertung_1"]),
    
    "HybridRAG": df_erg_manip["hybrid_rag_antwort_manipuliert_bewertung_1"] / 
                 (df_erg_manip["hybrid_rag_antwort_manipuliert_bewertung_0"] + df_erg_manip["hybrid_rag_antwort_manipuliert_bewertung_1"])
})

In [32]:
df_resilience_rate_by_type

,VectorRAG,GraphRAG,HybridRAG
fake_news_art,,,
Exaggeration,0.621053,0.331579,0.626316
Fake Fact,0.684211,0.278947,0.642105
Framing,0.615789,0.347368,0.647368
Summe,0.640351,0.319298,0.638596


**Gesamt:**

In [30]:
df_total = df_erg_manip.loc["Summe"]

total_resilience = pd.Series({
    "VectorRAG": df_total["vector_rag_antwort_manipuliert_bewertung_1"] / 
                 (df_total["vector_rag_antwort_manipuliert_bewertung_0"] + df_total["vector_rag_antwort_manipuliert_bewertung_1"]),
    
    "GraphRAG": df_total["graph_rag_antwort_manipuliert_bewertung_1"] / 
                (df_total["graph_rag_antwort_manipuliert_bewertung_0"] + df_total["graph_rag_antwort_manipuliert_bewertung_1"]),
    
    "HybridRAG": df_total["hybrid_rag_antwort_manipuliert_bewertung_1"] / 
                 (df_total["hybrid_rag_antwort_manipuliert_bewertung_0"] + df_total["hybrid_rag_antwort_manipuliert_bewertung_1"])
}).round(3) * 100


In [31]:
df_total

vector_rag_antwort_manipuliert_bewertung_0    205
vector_rag_antwort_manipuliert_bewertung_1    365
graph_rag_antwort_manipuliert_bewertung_0     388
graph_rag_antwort_manipuliert_bewertung_1     182
hybrid_rag_antwort_manipuliert_bewertung_0    206
hybrid_rag_antwort_manipuliert_bewertung_1    364
Name: Summe, dtype: int64

In [20]:
total_resilience_absolut = pd.Series({
    "VectorRAG": 365 / (205 + 365),
    "GraphRAG": 182 / (388 + 182),
    "HybridRAG": 364 / (206 + 364)
}).round(3) * 100

In [21]:
total_resilience_absolut

VectorRAG    64.0
GraphRAG     31.9
HybridRAG    63.9
dtype: float64

In [22]:
data = {
    "System": ["VectorRAG", "HybridRAG", "GraphRAG"],
    "Resilience Rate (%)": [64.0, 63.9, 31.9]
}
df_absoulute_res_ges = pd.DataFrame(data)

In [33]:
df_absoulute_res_ges

,System,Resilience Rate (%)
0,VectorRAG,64.0
1,HybridRAG,63.9
2,GraphRAG,31.9


## relative resilience Rate

**nur nach Systemen aufschlüsselbar** siehe BA

In [23]:
data_rel = {
    "System" :["GraphRAG","VectorRAG",  "HybridRAG"],
    " Relative Resilience Rate (%)": [100,80.0, 77.3]
}
df_relative_res = pd.DataFrame(data_rel)

# Erstellung der Diagramme mit Plotly express

**allgemeines Layout:**

In [25]:
 # Schaubilder erstellen

import plotly.express as px

df_plot_manip = df_erg_manip.drop(index="Summe")
# df in langes format
df_long = df_plot_manip.reset_index().melt(id_vars="fake_news_art", var_name="system_bewertung", value_name="anzahl")

df_long["system"] = df_long["system_bewertung"].str.extract(r"^(.*?)_bewertung")
df_long["bewertung"] = df_long["system_bewertung"].str.extract(r"_(\d)$")

df_vector = df_long[df_long["system"] == "vector_rag_antwort_manipuliert"]
df_graph = df_long[df_long["system"] == "graph_rag_antwort_manipuliert"]
df_hybrid = df_long[df_long["system"] == "hybrid_rag_antwort_manipuliert"]


layout_kwargs = dict(
    width=800,
    height=550,
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font=dict(size=22, color="#5F5F5F"),
    font=dict(size=16, color="#5F5F5F"),
    margin=dict(t=100),
    legend_title='Bewertung'
)

xaxis_kwargs = dict(ticks='', tickfont=dict(size=14))
yaxis_kwargs = dict(gridcolor='lightgrey', ticks='', tickfont=dict(size=14), )

color_map = {'0': "#B3B3B3", '1': "#096CB3"}

color_map_system = {
    "VectorRAG": "#1f77b4",
    "GraphRAG": "#ff7f0e",
    "HybridRAG": "#949494"
}



**Absolute Resilience Rate nach Systemen**


In [26]:


fig_ges_res = px.bar(
    df_absoulute_res_ges,
    x="Resilience Rate (%)",
    y="System",
    color="System",
    text="Resilience Rate (%)",
    title="Absolute Resilience Rate der Systeme",
    color_discrete_map=color_map_system,
    orientation="h"
)

fig_ges_res.update_layout(
    title=dict(text=fig_ges_res.layout.title.text, x=0, xanchor='left'),
    xaxis_title="Resilience Rate (%)",  
    yaxis_title="RAG-System",
    **layout_kwargs
)
fig_ges_res.update_xaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    range=[0, 70],
    **xaxis_kwargs
)

fig_ges_res.update_yaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    
    **yaxis_kwargs
)

fig_ges_res.update_traces(textposition="outside")

**Schaubild zur realtiven Resilience Rate der Systeme**

In [34]:
df_relative_res.columns = df_relative_res.columns.str.strip()

fig_ges_res_relativ = px.bar(
    df_relative_res,
    x="Relative Resilience Rate (%)",
    y="System",
    color="System",
    text="Relative Resilience Rate (%)",
    title="Relative Resilience Rate der Systeme",
    color_discrete_map=color_map_system,
    orientation="h",
)

fig_ges_res_relativ.update_layout(
    title=dict(text=fig_ges_res_relativ.layout.title.text, x=0, xanchor='left'),
    xaxis_title="Relative Resilience Rate (%)",  
    yaxis_title="RAG-System",
    **layout_kwargs
)
fig_ges_res_relativ.update_xaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    range=[0, 110],
    **xaxis_kwargs
)

fig_ges_res_relativ.update_yaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    
    **yaxis_kwargs
)

fig_ges_res_relativ.update_traces(textposition="outside")

**Schaubild zur Absoluten Resilience Rate nach der Fake News Art:**

In [ ]:



df_long = df_resilience_rate_by_type.drop("Summe").reset_index().melt(id_vars="fake_news_art", 
    var_name="System", value_name="Resilience Rate")

# Prozentwerte transformieren
df_long["Resilience Rate (%)"] = (df_long["Resilience Rate"] * 100).round(1)

df_sorted = (
    df_long.sort_values(["Resilience Rate (%)"], ascending=[True])
    .copy()
)




# Plot erstellen
fig_resilience = px.bar(
    df_sorted,
    x="Resilience Rate (%)",
    y="fake_news_art",
    color="System",
    barmode="group",
    title="Absolute Resilience Rate der Systeme nach Fake-News-Art",
    text="Resilience Rate (%)",
    color_discrete_map=color_map_system,
    orientation="h",
    category_orders={"fake_news_art": ["Fake Fact", "Exaggeration", "Framing"]}
)

# Layout-Einstellungen
fig_resilience.update_layout(
    title=dict(text=fig_resilience.layout.title.text, x=0, xanchor='left'),
    bargroupgap=0.1,
    xaxis_title="Resilience Rate (%)",  
    yaxis_title="Fake-News-Art",
    **layout_kwargs
)

fig_resilience.update_xaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    range=[0, 100],  
    **xaxis_kwargs
)

fig_resilience.update_yaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    **yaxis_kwargs
)

fig_resilience.update_traces(textposition="outside")

fig_resilience.show()


**Schaubild zu der Häufigkeit der Bewertungsausprägungen im VectorRAG-System, aufgeschlüsselt nach Fake News Arten:**

In [27]:



# Diagramm für VectorRAG System


df_vector_sorted = df_vector.sort_values(by="anzahl")
max_val_vector = df_vector["anzahl"].max()

fig_vector = px.bar(
    df_vector_sorted,
    y="fake_news_art",
    x="anzahl",
    color="bewertung",
    color_discrete_map=color_map,
    barmode="group",
    title="Bewertung VectorRAG-System nach Fake-News-Art",
    text="anzahl",
    orientation="h",
    category_orders={"fake_news_art": ["Fake Fact", "Exaggeration", "Framing"]}
)


fig_vector.update_layout(
    title=dict(text=fig_vector.layout.title.text, x=0, xanchor='left'),
    bargroupgap=0.1,
    xaxis_title="Anzahl",  
    yaxis_title="Fake-News-Art",
    **layout_kwargs
)

fig_vector.update_xaxes(
    showgrid=False,       
    showline=True,   
    linecolor='#5F5F5F',
    range=[0, max_val_vector * 1.1],  
    **xaxis_kwargs
)

fig_vector.update_yaxes(
    showgrid=False,       
    showline=True,        
    linecolor='#5F5F5F',
    **yaxis_kwargs
)

fig_vector.update_traces(textposition="outside")




**Schaubild zu der Häufigkeit der Bewertungsausprägungen im GraphRAG-System, aufgeschlüsselt nach Fake News Arten:**

In [28]:
# Diagramm für GraphRAG System

df_graph_sorted = df_graph.sort_values(by="anzahl", ascending=False)
max_val_graph = df_graph["anzahl"].max()

fig_graph = px.bar(
    df_graph_sorted,
    y="fake_news_art",
    x="anzahl",
    color="bewertung",
    color_discrete_map=color_map,
    barmode="group",
    title="Bewertung GraphRAG-System nach Fake-News-Art",
    text="anzahl",
    orientation="h",
    category_orders={"fake_news_art": ["Fake Fact", "Exaggeration", "Framing"]}
)


fig_graph.update_layout(
    title=dict(text=fig_graph.layout.title.text, x=0, xanchor='left'),
    bargroupgap=0.1,
    xaxis_title="Anzahl",  
    yaxis_title="Fake-News-Art",
    **layout_kwargs
)

fig_graph.update_xaxes(
    showgrid=False,       
    showline=True,   
    linecolor='#5F5F5F',  
    range=[0, max_val_graph * 1.1],
    **xaxis_kwargs
)

fig_graph.update_yaxes(
    showgrid=False,       
    showline=True,        
    linecolor='#5F5F5F',
    **yaxis_kwargs
)

fig_graph.update_traces(textposition="outside")

**Schaubild zu der Häufigkeit der Bewertungsausprägungen im HybridRAG-System, aufgeschlüsselt nach Fake News Arten:**

In [35]:
# Diagramm für HybridRAG

df_hybrid_sorted = df_hybrid.sort_values(by="anzahl")
max_val_hybrid = df_hybrid["anzahl"].max()

fig_hybrid = px.bar(
    df_hybrid_sorted,
    y="fake_news_art",
    x="anzahl",
    color="bewertung",
    color_discrete_map=color_map,
    barmode="group",
    title="Bewertung HybridRAG-System nach Fake-News-Art",
    text="anzahl",
    orientation="h",
    category_orders={"fake_news_art": ["Fake Fact", "Exaggeration", "Framing"]}
)


fig_hybrid.update_layout(
    title=dict(text=fig_hybrid.layout.title.text, x=0, xanchor='left'),
    bargroupgap=0.1,
    xaxis_title="Anzahl",  
    yaxis_title="Fake-News-Art",
    **layout_kwargs
)

fig_hybrid.update_xaxes(
    showgrid=False,       
    showline=True,   
    linecolor='#5F5F5F',  
    range=[0, max_val_hybrid * 1.1],
    **xaxis_kwargs
)

fig_hybrid.update_yaxes(
    showgrid=False,       
    showline=True,        
    linecolor='#5F5F5F',
    **yaxis_kwargs
)

fig_hybrid.update_traces(textposition="outside")

**Schaubild zu der Häufigkeit der Bewertungsausprägungen Für jedes System im manipulierten Kontext**

In [36]:
# Summe berechnen
df_sum = df_plot_manip.sum().reset_index()
df_sum.columns = ["system_bewertung", "anzahl"]  # explizit Spalten benennen

# Systemnamen und Bewertungen extrahieren
df_sum["system"] = df_sum["system_bewertung"].str.extract(r"^(.*?)_bewertung")
df_sum["bewertung"] = df_sum["system_bewertung"].str.extract(r"_(\d)$")

# Systemnamen umbenennen
system_label_map = {
    "vector_rag_antwort_manipuliert": "VectorRAG",
    "graph_rag_antwort_manipuliert": "GraphRAG",
    "hybrid_rag_antwort_manipuliert": "HybridRAG"
}
df_sum["system"] = df_sum["system"].map(system_label_map)


df_sum_sorted = df_sum.sort_values(by="anzahl")
max_val_sum = df_sum["anzahl"].max()


fig_total = px.bar(
    df_sum,
    y="system",
    x="anzahl",
    color="bewertung",
    barmode="group",
    color_discrete_map=color_map,
    title="Gesamte Bewertungen pro System manipulierter Kontext",
    text="anzahl",
    orientation="h",
    category_orders={"system": ["VectorRAG", "HybridRAG", "GraphRAG"]}
)

fig_total.update_layout(
    title=dict(text=fig_total.layout.title.text, x=0, xanchor='left'),
    bargroupgap=0.1,
    xaxis_title="Anzahl",  
    yaxis_title="System",
    **layout_kwargs
)



fig_total.update_xaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    range=[0, max_val_sum * 1.1],  # Platz für Text über Balken
    **xaxis_kwargs
)

fig_total.update_yaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    **yaxis_kwargs
)

fig_total.update_traces(textposition="outside")

**Schaubild zu der Häufigkeit der Bewertungsausprägungen Je System im Originalkontext:**

In [13]:
# original
# Summe berechnen
df_plot_orig = df_erg_org.drop(index="Summe")
df_sum = df_plot_orig.sum().reset_index()
df_sum.columns = ["system_bewertung", "anzahl"] 





# Systemnamen und Bewertungen extrahieren
df_sum["system"] = df_sum["system_bewertung"].str.extract(r"^(.*?)_bewertung")
df_sum["bewertung"] = df_sum["system_bewertung"].str.extract(r"_(\d)$")

# Systemnamen umbenennen
system_label_map = {
    "vector_rag_antwort_original": "VectorRAG",
    "graph_rag_antwort_original": "GraphRAG",
    "hybrid_rag_antwort_original": "HybridRAG"
}
df_sum["system"] = df_sum["system"].map(system_label_map)

# Danach kannst du wie geplant fortfahren:
df_sum_sorted = df_sum.sort_values(by="anzahl")
max_val_sum = df_sum["anzahl"].max()


fig_total_ori = px.bar(
    df_sum,
    y="system",
    x="anzahl",
    color="bewertung",
    barmode="group",
    color_discrete_map=color_map,
    title="Gesamte Bewertungen pro System Originalkontext",
    text="anzahl",
    orientation="h",
    category_orders={"system": ["HybridRAG", "VectorRAG", "GraphRAG"]}
)

fig_total_ori.update_layout(
    title=dict(text=fig_total_ori.layout.title.text, x=0, xanchor='left'),
    bargroupgap=0.1,
    xaxis_title="Anzahl",  
    yaxis_title="System",
    **layout_kwargs
)



fig_total_ori.update_xaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    range=[0, max_val_sum * 1.1],  # Platz für Text über Balken
    **xaxis_kwargs
)

fig_total_ori.update_yaxes(
    showgrid=False,
    showline=True,
    linecolor='#5F5F5F',
    **yaxis_kwargs
)

fig_total_ori.update_traces(textposition="outside")